# 데이터셋 선택 및 하이퍼파라미터 설정
▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼

In [1]:
import time
import natsort
import os

folder_list = os.listdir("./data")
item_list = natsort.natsorted(folder_list)

print("다음 데이터셋들이 학습됩니다 : ", item_list)

다음 데이터셋들이 학습됩니다 :  ['bottle', 'cable', 'capsule', 'carpet', 'grid', 'hazelnut', 'leather', 'metal_nut', 'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper']


In [2]:
#최소10, 200~400 추천, 10단위로 pth가 저장됨
epochs = 200
batch_size = 16

#3090 24GB에서 64까지 사용 가능했음
learning_rate = 0.005
image_size = 256

In [3]:
import torch
from dataset import get_data_transforms
from torchvision.datasets import ImageFolder
import numpy as np
import random
import os
from torch.utils.data import DataLoader
from ConvNext_V1_RD_study import convnext_base
from OCE import BN_layer, AttnBottleneck
from dataset import RD_Dataset
from de_ConvNext import de_convnext_base
import torch.backends.cudnn as cudnn
import argparse
from torch.nn import functional as F

torch.cuda.is_available()

True

In [4]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
def loss_fucntion(a, b):
    #mse_loss = torch.nn.MSELoss()
    cos_loss = torch.nn.CosineSimilarity()
    loss = 0
    for item in range(len(a)):
        #print(a[item].shape)
        #print(b[item].shape)
        #loss += 0.1*mse_loss(a[item], b[item])
        loss += torch.mean(1-cos_loss(a[item].view(a[item].shape[0],-1),
                                      b[item].view(b[item].shape[0],-1)))
    return loss

In [6]:
def loss_concat(a, b):
    mse_loss = torch.nn.MSELoss()
    cos_loss = torch.nn.CosineSimilarity()
    loss = 0
    a_map = []
    b_map = []
    size = a[0].shape[-1]
    for item in range(len(a)):
        #loss += mse_loss(a[item], b[item])
        a_map.append(F.interpolate(a[item], size=size, mode='bilinear', align_corners=True))
        b_map.append(F.interpolate(b[item], size=size, mode='bilinear', align_corners=True))
    a_map = torch.cat(a_map,1)
    b_map = torch.cat(b_map,1)
    loss += torch.mean(1-cos_loss(a_map,b_map))
    return loss

In [7]:
encoder = convnext_base(pretrained=True)


c:\Users\asiclab\anaconda3\envs\RD\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\asiclab\anaconda3\envs\RD\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Base_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Base_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


pretrained weights have been successfully loaded


In [8]:
decoder = de_convnext_base(weights=False)

c:\Users\asiclab\anaconda3\envs\RD\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ConvNeXt_Base_Weights.verify <bound method WeightsEnum.verify of <enum 'ConvNeXt_Base_Weights'>>


In [11]:

bn = BN_layer(AttnBottleneck,3)

In [ ]:
def train(_class_):
    print(_class_)
        
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)

    data_transform = get_data_transforms(image_size, image_size)
    
    train_path = './data/' + _class_ + '/train'
    ckp_path = './checkpoints/' + 'wres50_'+_class_+'.pth'
    
    train_data = ImageFolder(root=train_path, transform=data_transform)
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # encoder, bn = wide_resnet50_2(pretrained=True)
    encoder = convnext_base(pretrained=True)
    bn = BN_layer(AttnBottleneck,3)
    encoder = encoder.to(device)
    bn = bn.to(device)
    encoder.eval()
    decoder = de_convnext_base(pretrained=False)
    decoder = decoder.to(device)

    optimizer = torch.optim.Adam(list(decoder.parameters())+list(bn.parameters()), lr=learning_rate, betas=(0.5,0.999))


    for epoch in range(epochs):
        start = time.time() 
        
        bn.train()
        decoder.train()
        loss_list = []
        for img, label in train_dataloader:
            img = img.to(device)
            inputs = encoder(img)
            outputs = decoder(bn(inputs))#bn(inputs))
            loss = loss_fucntion(inputs, outputs)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, epochs, np.mean(loss_list)))
        print("time :",time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
        
        if (epoch + 1) % 10 == 0:
            torch.save({'bn': bn.state_dict(),'decoder': decoder.state_dict()}, ckp_path)
            
    return loss

In [ ]:
#학습
for i in item_list:
    start_class = time.time()  # 시작 시간 저장

    train(i)
    print(i, "time :",time.time() - start_class)  # 현재시각 - 시작시간 = 실행 시간